In [ ]:
#Step 1: Install Required Libraries
!pip install faiss-cpu scann datasketch sentence-transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [ ]:
#Step 2: Import Libraries
import random
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sentence_transformers import SentenceTransformer
import faiss
from datasketch import MinHash, MinHashLSH
import scann


In [ ]:
#Step 3: Generate Mock Amazon Product Dataset
categories = ["Smartphone", "Laptop", "Headphones", "Smartwatch", "Camera", "Tablet", "Speaker"]
features = [
    "with 4K display", "with long battery life", "with noise cancellation", "with stylus support",
    "with AI features", "with water resistance", "with fast charging", "with dual SIM",
    "with 5G connectivity", "with touchscreen", "with fingerprint sensor", "with triple camera"
]

def generate_product():
    return f"{random.choice(categories)} {random.choice(features)}"

product_texts = [generate_product() for _ in range(500)]
df = pd.DataFrame({'product_description': product_texts})
df.head()

,product_description
0,Camera with noise cancellation
1,Smartwatch with water resistance
2,Laptop with stylus support
3,Headphones with stylus support
4,Laptop with noise cancellation


In [ ]:
#Step 4: Generate Embeddings
# Load sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for all product descriptions
embeddings = model.encode(df['product_description'].tolist(), show_progress_bar=True)

# Convert to numpy array
embeddings = np.array(embeddings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [ ]:
#Step 5: Define a Query and Generate Its Embedding

# Define a test query
query_text = "Smartphone with fast charging"
query_embedding = model.encode([query_text])[0]


In [ ]:
#Step 6: Build Search Indexes for FAISS, LSH, and ScaNN
#6.1 — FAISS (Flat L2 Index)
dimension = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(embeddings)

#6.2 — MinHash LSH (Jaccard Similarity)
# Convert product descriptions to sets of shingles
def get_minhash(text, num_perm=128):
    m = MinHash(num_perm=num_perm)
    for word in text.lower().split():
        m.update(word.encode('utf8'))
    return m

# Create LSH index
lsh_index = MinHashLSH(threshold=0.3, num_perm=128)
minhashes = []

for i, desc in enumerate(df['product_description']):
    m = get_minhash(desc)
    lsh_index.insert(f"item_{i}", m)
    minhashes.append(m)

# Create MinHash for query
query_minhash = get_minhash(query_text)

#6.3 — ScaNN (Cosine Similarity)
# Create ScaNN searcher (cosine distance)
scann_searcher = scann.scann_ops_pybind.builder(embeddings, 10, "dot_product")\
    .score_ah(2, anisotropic_quantization_threshold=0.2)\
    .reorder(10)\
    .build()



In [ ]:
#Step 7: Perform Search + Benchmark Results
# Store results and timing
results = {}
# FAISS Search
start = time.time()
_, faiss_indices = faiss_index.search(np.array([query_embedding]), k=5)
results['faiss'] = [df['product_description'].iloc[i] for i in faiss_indices[0]]
faiss_time = time.time() - start

#  MinHash LSH Search
start = time.time()
lsh_result_keys = lsh_index.query(query_minhash)
results['lsh'] = [df['product_description'].iloc[int(key.split("_")[1])] for key in lsh_result_keys]
lsh_time = time.time() - start

# ScaNN Search
start = time.time()
scann_indices, _ = scann_searcher.search(query_embedding)  # Pass 1D vector directly
results['scann'] = [df['product_description'].iloc[i] for i in scann_indices]
scann_time = time.time() - start


In [ ]:
# Print Timing Summary
print("Search Time (seconds):")
print(f"FAISS:  {faiss_time:.4f}s")
print(f"LSH:    {lsh_time:.4f}s")
print(f"ScaNN:  {scann_time:.4f}s")

# Show Results
for method, res in results.items():
    print(f"\n{method.upper()} Results:")
    for r in res:
        print(" -", r)


Search Time (seconds):
FAISS:  0.0009s
LSH:    0.0017s
ScaNN:  0.0067s

FAISS Results:
 - Smartphone with fast charging
 - Smartphone with fast charging
 - Smartphone with fast charging
 - Smartphone with fast charging
 - Smartphone with fast charging

LSH Results:
 - Camera with dual SIM
 - Smartphone with triple camera
 - Tablet with fingerprint sensor
 - Speaker with triple camera
 - Smartwatch with fast charging
 - Camera with long battery life
 - Speaker with 4K display
 - Smartphone with touchscreen
 - Camera with dual SIM
 - Headphones with noise cancellation
 - Tablet with dual SIM
 - Speaker with triple camera
 - Camera with dual SIM
 - Camera with long battery life
 - Speaker with long battery life
 - Speaker with dual SIM
 - Tablet with triple camera
 - Speaker with dual SIM
 - Smartphone with touchscreen
 - Camera with 4K display
 - Headphones with fingerprint sensor
 - Speaker with long battery life
 - Camera with triple camera
 - Tablet with long battery life
 - Headphone